In [ ]:
import torch

In [ ]:
from tqdm.notebook import tqdm  #shows how far forloop is gone (visually)
import pandas as pd

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 895 kB 33.2 MB/s 
     |████████████████████████████████| 3.3 MB 37.2 MB/s 
     |████████████████████████████████| 596 kB 32.7 MB/s 
     |████████████████████████████████| 59 kB 6.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving applestore_data_cleaned.csv to applestore_data_cleaned.csv


In [ ]:
import io
df = pd.read_csv('applestore_data_cleaned.csv')

In [ ]:
test_df = pd.read_csv("testing_data.csv")

In [ ]:
#df = pd.read_csv("applestore_data_merged.csv")
#df = df[df.category != "Games"]

In [ ]:
possible_labels = df.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Book': 20,
 'Business': 11,
 'Catalogs': 22,
 'Education': 16,
 'Entertainment': 13,
 'Finance': 5,
 'Food & Drink': 18,
 'Games': 0,
 'Health & Fitness': 12,
 'Lifestyle': 17,
 'Medical': 21,
 'Music': 6,
 'Navigation': 15,
 'News': 19,
 'Photo & Video': 14,
 'Productivity': 1,
 'Reference': 4,
 'Shopping': 3,
 'Social Networking': 9,
 'Sports': 10,
 'Travel': 8,
 'Utilities': 7,
 'Weather': 2}

In [ ]:
df.category.value_counts() #.sort_values().plot(kind="barh")

Games                3392
Entertainment         442
Education             404
Photo & Video         341
Utilities             201
Productivity          167
Health & Fitness      157
Music                 134
Social Networking     116
Sports                 99
Lifestyle              97
Shopping               83
Weather                65
Book                   60
Travel                 53
Reference              52
Business               52
News                   49
Finance                45
Food & Drink           42
Navigation             32
Medical                21
Catalogs                6
Name: category, dtype: int64

In [ ]:
df['label'] = df.category.replace(label_dict)

In [ ]:
test_df['label'] = test_df.category_2.replace(label_dict)

In [ ]:
df

,Unnamed: 0,Unnamed: 0.1,id,app_name,app_desc,category,language,category1,app_desc_clean,app_name_clean,app_name_desc_clean,cat1,label,name_desc
0,0,0,281656475,PAC-MAN Premium,"SAVE 20%, now only $3.99 for a limited time!\n...",Games,en,GAME,save limited time one popular video games arca...,pac man premium,pac man premium save limited time one popular ...,0,0,"PAC-MAN Premium SAVE 20%, now only $3.99 for a..."
1,1,1,281796108,Evernote - stay organized,Let Evernote change the way you organize your ...,Productivity,en,APP,let evernote change way organize personal prof...,evernote organized,evernote organized let evernote change way org...,1,1,Evernote - stay organized Let Evernote change ...
2,2,2,281940292,"WeatherBug - Local Weather, Radar, Maps, Alerts",Download the most popular free weather app pow...,Weather,en,APP,popular weather powered largest professional w...,weatherbug local weather radar maps alerts,weatherbug local weather radar maps alerts pop...,1,2,"WeatherBug - Local Weather, Radar, Maps, Alert..."
3,3,3,282614216,"eBay: Best App to Buy, Sell, Save! Online Shop...",The eBay app is the best way to find anything ...,Shopping,en,APP,ebay best way find anything need dresses fashi...,ebay best buy sell save shopping,ebay best buy sell save shopping ebay best way...,1,3,"eBay: Best App to Buy, Sell, Save! Online Shop..."
4,4,4,282935706,Bible,On more than 250 million devices around the wo...,Reference,en,APP,million devices around world people reading li...,bible,bible million devices around world people read...,1,4,Bible On more than 250 million devices around ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6105,7192,7192,1187617475,Kubik,Place the falling blocks correctly in order to...,Games,en,GAME,place falling blocks correctly order clear lay...,kubik,kubik place falling blocks correctly order cle...,0,0,Kubik Place the falling blocks correctly in or...
6106,7193,7193,1187682390,VR Roller-Coaster,A thrilling virtual reality roller coaster exp...,Games,en,GAME,thrilling virtual reality roller coaster exper...,vr roller coaster,vr roller coaster thrilling virtual reality ro...,0,0,VR Roller-Coaster A thrilling virtual reality ...
6107,7194,7194,1187779532,Bret Michaels Emojis + Lyric Keyboard,"Rock star Bret Michaels, winner of Celebrity A...",Utilities,en,APP,rock star bret winner celebrity apprentice cha...,bret michaels emojis lyric keyboard,bret michaels emojis lyric keyboard rock star ...,1,7,Bret Michaels Emojis + Lyric Keyboard Rock sta...
6108,7195,7195,1187838770,VR Roller Coaster World - Virtual Reality,VR Roller Coaster World is an app for Google C...,Games,en,GAME,vr roller coaster world google cardboard creat...,vr roller coaster world virtual reality,vr roller coaster world virtual reality vr rol...,0,0,VR Roller Coaster World - Virtual Reality VR R...


In [ ]:
df["name_desc"] = df["app_name"] + " " + df["app_desc"]

X_train = df[df["label"] != 0].name_desc
y_train = df[df["label"] != 0].label

X_val = test_df[test_df["label"] != 0].user_query
y_val = test_df[test_df["label"] != 0].label

In [ ]:
test_df

,user_query,category_2,category_1,label
0,fart sounds,Entertainment,1,13
1,funny face filter,Entertainment,1,13
2,emoji maker,Entertainment,1,13
3,fake gps,Entertainment,1,13
4,minecraft cheats,Entertainment,1,13
...,...,...,...,...
176,call of duty,Games,0,0
177,pokemon go,Games,0,0
178,roblox,Games,0,0
179,brawl stars,Games,0,0


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(  #can tale multiple strings and encode them as we need them
    X_train.values, 
    add_special_tokens=True, 
    return_attention_mask=True, #binary mask indicating where model should attend to [1,1,1,0,0] 0 = padded instances
    pad_to_max_length=True, 
    max_length=300, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    X_val.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=300, 
    return_tensors='pt'
)

#ENCODED VALS IS A DICTIONARY - need to access that dictionary and pull out what we need

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(y_train.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(y_val.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train) #standard way of using dataset in pytorch
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
len(dataset_train)

2718

In [ ]:
dataset_train[102]  #all correct - the title, the attention mask and the label

(tensor([  101,  3712,  2739,  3712,  2739,  1011,  2034,  2005,  4911,  2739,
          1996,  3712,  2739, 10439,  2038,  2042, 25416, 21898,  2098,  2000,
          3288,  2017,  1037,  4435,  2047,  2298,  1998,  2514,  1010,  2488,
          2836,  1998,  1037, 26108,  3325,  2043, 10523,  2256,  4180,  1012,
          3145,  2838,  1024,  1528,  4374, 26828,  2015,  1997,  4911,  2739,
          1528,  3422,  3712,  2739,  2444,  2013,  5973,  1999,  1996,  2866,
          1528,  3193,  2035,  1996,  6745,  2327,  3441,  2030, 11347,  2063,
          4180,  2011,  2930,  1528,  3191,  1999,  1011,  5995,  4180,  2517,
          2011,  3712,  2739, 11370,  2015,  1528,  3193,  1996,  2200,  2190,
          3712,  2739,  6876,  2058, 15536,  8873,  1998,  4684,  2897,  1528,
          3193,  3746, 11726,  1999,  1996, 10439,  1528,  3745,  3441,  2006,
          2591,  2865,  1528,  3613,  3752,  3441,  2125,  4179,  3531,  3602,
          1024,  3666,  2678,  2089,  4297,  3126,  

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 10   #set to more when more hardware available e.g. 100

dataloader_train = DataLoader(dataset_train,      #iterators to iterate through dataset in batches
                              sampler=RandomSampler(dataset_train), #sampling randomly from training set
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), #doesnt matter here so can be sequential
                                   batch_size=batch_size)  #validation set smaller so batch size could be bigger

In [ ]:
len(dataloader_train)

272

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=2e-5, #2e-5 based on original bert paper
                  eps=1e-8)  #this is default
                  
epochs = 2  
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)  
#batch size is 3 so 2130/3 = 710 batches * epochs = training steps - pretty obvious

In [ ]:
from sklearn.metrics import f1_score

#preds = [0.9, 0.05, 0, 0.05, 0]
#but we want preds = [1, 0 ,0 ,0 ,0]

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted') #class distrbution taken into account hance weighted, ca use macro too

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')


In [ ]:
import numpy as np

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


# New section

In [ ]:


def evaluate(dataloader_val):

    model.eval()  #evaluation mode- no backpropagation, eval mode freezes all weights
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():       #no gradients 
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()  #in case using GPU you want to pull them into CPU to use numpy
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    


In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0 #adding each batches loss to this

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()  #only keep gradients for recurrent NN
       
       
        batch = tuple(b.to(device) for b in batch) #only important for GPU
        
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs) #unpacks dictionary into inputs https://stackoverflow.com/questions/36901/what-does-double-star-asterisk-and-star-asterisk-do-for-parameters
        
        loss = outputs[0]   #outputs is loss and logits, logits = outputs[1]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   #prevent exploding or vanishing gradients

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/272 [00:00<?, ?it/s]


Epoch 1
Training loss: 2.149518975001924
Validation loss: 2.4561465978622437
F1 Score (Weighted): 0.2333902980152306


Epoch 2:   0%|          | 0/272 [00:00<?, ?it/s]


Epoch 2
Training loss: 1.2807994201121962
Validation loss: 2.33536865494468
F1 Score (Weighted): 0.2863825012907582


note: loss is NOT a percentage 
https://stackoverflow.com/questions/34518656/how-to-interpret-loss-and-accuracy-for-a-machine-learning-model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('finetuned_BERT_epoch_2.model', map_location=torch.device('cpu')))
print("model loaded")

_, predictions, true_vals = evaluate(dataloader_validation) # preferably have a test set
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

model loaded
Class: Productivity
Accuracy: 0/6

Class: Weather
Accuracy: 4/5

Class: Shopping
Accuracy: 0/5

Class: Reference
Accuracy: 0/3

Class: Finance
Accuracy: 0/7

Class: Music
Accuracy: 0/7

Class: Utilities
Accuracy: 2/7

Class: Travel
Accuracy: 0/6

Class: Social Networking
Accuracy: 3/7

Class: Sports
Accuracy: 3/7

Class: Health & Fitness
Accuracy: 0/8

Class: Entertainment
Accuracy: 10/10

Class: Photo & Video
Accuracy: 5/9

Class: Navigation
Accuracy: 0/2

Class: Education
Accuracy: 7/9

Class: Lifestyle
Accuracy: 0/4

Class: Food & Drink
Accuracy: 0/3

Class: Book
Accuracy: 0/4



In [ ]:
f1_score_func(predictions, true_vals)

0.2863825012907582

In [ ]:
predictions